In [1]:
from api.sync import Sync

client = Sync('http://192.168.99.102/vtigercrm_2022', 'superadmin', 'MFaeyxCMTmRrUZiE')

service_id = "24x3"
product_id = "14x11"
assigned_user_id = "19x1"

19x113


In [2]:
import requests

# Datos de la API
base_url = "http://development.localhost:8000"
api_key = "8d2f08387ea6aca"
api_secret = "9b887566130a4cd"

# Headers para autenticación
headers = {
    "Authorization": f"token {api_key}:{api_secret}"
}

# Endpoint que deseas consumir (ejemplo: listar documentos de la Doctype 'Item')
endpoint = "/api/method/frappe.auth.get_logged_user"

# Hacer la petición GET
response = requests.get(f"{base_url}{endpoint}", headers=headers)

# Ver el resultado
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code} - {response.text}")

{'message': 'Administrator'}


In [3]:
module = "SalesOrder"
condition = {
    "cf_2059" :"2025-01-01"
}

module_object = client.search(module, condition)
print(module_object[0])

{'salesorder_no': 'SO17305', 'subject': 'KELVIN CEDENO MORA', 'potential_id': '', 'customerno': '', 'quote_id': '', 'vtiger_purchaseorder': '', 'contact_id': '12x26532', 'duedate': '', 'carrier': '', 'pending': '', 'sostatus': '', 'txtAdjustment': '0.00000000', 'salescommission': '', 'exciseduty': '', 'hdnGrandTotal': '0.00000000', 'hdnSubTotal': '0.00000000', 'hdnTaxType': 'group', 'discount_percent': '', 'discount_amount': '', 'hdnS_H_Amount': '0.00000000', 'account_id': '', 'assigned_user_id': '19x113', 'createdtime': '2023-11-06 15:25:13', 'modifiedtime': '2024-09-09 19:24:03', 'modifiedby': '19x204', 'currency_id': '21x1', 'conversion_rate': '1.000', 'enable_recurring': '0', 'recurring_frequency': '', 'start_period': '', 'end_period': '', 'payment_duration': '', 'invoicestatus': '', 'productid': '14x29415', 'quantity': '1.000', 'listprice': '0.00000000', 'comment': '', 'tax1': '', 'tax2': '', 'tax3': '', 'pre_tax_total': '0.00000000', 'hdnS_H_Percent': '0', 'cf_1463': 'YES', 'cf_1

In [4]:
import json
salesorder = module_object[10]
#print(json.dumps(salesorder, indent=4))

In [5]:
contact = client.get_contact_from_id(salesorder['contact_id'])
print(contact)

{'firstname': 'MARIA', 'middlename': 'ANTONIA', 'lastname': 'PENA DUARTE', 'ssn': '', 'dob': '1989-06-04', 'gender': 'FEMALE', 'phone_1': '4074702750', 'phone_2': '', 'email': 'maria.a.pena04@hotmail.com'}


In [34]:
contact_data = {
    "first_name": contact['firstname'],
    "middle_name": contact['middlename'],
    "last_name": contact['lastname'],
    "email_ids": [  # Child table para emails
        {
            "email_id": contact['email'],  # Email principal
            "is_primary": 1  # Marca como principal
        }
    ],
    "phone_nos": [  # Child table para números de teléfono
        {
            "phone": contact['phone_1'],  # Teléfono principal
            "is_primary": 1  # Marca como principal
        },
        {
            "phone": contact['phone_2'],  # Teléfono principal
            "is_primary": 0  # Marca como principal
        }
    ]
}

address_data = {
    "address_line1": salesorder['cf_2737'],    # Dirección línea 1
    "city": salesorder['cf_2739'],
    "state": salesorder['state'],
    "zipcode": salesorder['zipcode'],
    "country": "United States",
}

In [ ]:
endpoint = "/api/resource/Customer"

customer_data = {
    "customer_name": f"{contact_data['first_name']} {contact_data['middle_name']} {contact_data['last_name']}",  # Nombre completo del cliente o empresa
    "customer_type": "Individual",
    "territory": "All Territories"
}

response = requests.post(f"{base_url}{endpoint}", json=customer_data, headers=headers)

customer_name = None
# Ver el resultado
if response.status_code == 200:
    print("Contacto creado exitosamente:")
    resp = response.json()
    print(resp)
    customer_name = resp['data']['name']
else:
    print(f"Error al crear el contacto: {response.status_code} - {response.text}")

customer_name

In [ ]:
endpoint = "/api/resource/Contact"

contact_data['links'] = [
    {
        "link_doctype": "Customer",
        "link_name": customer_name  # Vinculamos el contacto con el Customer creado
    }
]
response = requests.post(f"{base_url}{endpoint}", json=contact_data, headers=headers)

contact_name = None
# Ver el resultado
if response.status_code == 200:
    print("Contacto creado exitosamente:")
    resp = response.json()
    contact_name = resp['data']['name']
else:
    print(f"Error al crear el contacto: {response.status_code} - {response.text}")

contact_name

In [ ]:
endpoint = "/api/resource/Address"

# Datos del nuevo contacto
address_data["address_title"] = f"{customer_name}"  # Título de la dirección
address_data['links'] = [
    {
        "link_doctype": "Customer",
        "link_name": customer_name  # Vinculamos el contacto con el Customer creado
    }
]

response = requests.post(f"{base_url}{endpoint}", json=address_data, headers=headers)

address_name = None
# Ver el resultado
if response.status_code == 200:
    print("Dirección creada exitosamente:")
    resp = response.json()
    print(resp)
    address_name = resp['data']['name']
else:
    print(f"Error al crear la dirección: {response.status_code} - {response.text}")

address_name

In [ ]:
update_data = {
    "customer_primary_contact": contact_name,  # ID del contacto como primario
    "customer_primary_address": address_name  # Reemplaza con el ID de la dirección primaria
}

update_customer_endpoint = f"/api/resource/Customer/{customer_name}"

update_response = requests.put(f"{base_url}{update_customer_endpoint}", json=update_data, headers=headers)

# Ver el resultado de la actualización del Cliente
if update_response.status_code == 200:
    print("Cliente actualizado exitosamente:", update_response.json())
else:
    print(f"Error al actualizar el cliente: {update_response.status_code} - {update_response.text}")